In [12]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

In [14]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [15]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-05-14/"
files_start = 2336
files_end = 2337


In [16]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [17]:
filenames = os.listdir(path_all)

In [18]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [19]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2016-05-14 17:25:00,1,blank sol,0,0,0.000695,0.000524,0.004266,0.000969,0.002686,0.000787,...,0.000073,2.997210e-05,0.000769,0.000024,0.000576,0.000076,0.004659,0.001018,0.002957,0.000967
2016-05-14 17:25:00,2,blank sol,0,0,0.000827,0.000668,0.005110,0.001175,0.003311,0.001019,...,0.000062,6.393900e-06,0.000962,0.000067,0.000711,0.000073,0.005736,0.001335,0.003714,0.001186
2016-05-14 17:25:00,3,blank sol,0,0,0.000735,0.000528,0.004391,0.000973,0.002748,0.000851,...,0.000066,3.211430e-05,0.000790,0.000062,0.000615,0.000063,0.004886,0.001109,0.003120,0.001023
2016-05-14 17:25:00,4,blank sol,0,0,0.000977,0.000745,0.005890,0.001378,0.003826,0.001161,...,0.000030,4.245000e-06,0.001020,0.000044,0.000783,0.000079,0.006015,0.001422,0.003913,0.001289
2016-05-14 17:25:00,5,blank sol,0,0,0.002236,0.001678,0.012405,0.003053,0.008547,0.002686,...,0.000026,1.847250e-05,0.000748,0.000074,0.000571,0.000066,0.004558,0.001025,0.002902,0.000908
2016-05-14 17:25:00,6,blank sol,0,0,0.000725,0.000560,0.004453,0.000996,0.002848,0.000904,...,0.000046,-2.890000e-08,0.000881,0.000044,0.000670,0.000071,0.005224,0.001203,0.003352,0.001065
2016-05-14 17:25:00,7,blank sol,0,0,0.000688,0.000521,0.004251,0.000914,0.002646,0.000832,...,0.000013,4.254500e-06,0.000654,0.000055,0.000529,0.000074,0.004030,0.000886,0.002476,0.000794
2016-05-14 17:25:00,8,blank sol,0,0,0.000645,0.000487,0.003980,0.000843,0.002462,0.000780,...,0.000035,4.261400e-06,0.000716,0.000077,0.000557,0.000085,0.004460,0.001013,0.002794,0.000916
2016-05-14 17:25:00,9,blank sol,0,0,0.000851,0.000649,0.005176,0.001179,0.003296,0.001035,...,0.000039,1.567950e-05,0.000777,0.000048,0.000653,0.000062,0.004865,0.001113,0.003088,0.000990


In [21]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2016-05-14 12:05:00,0,0,0.001002,0.000757,0.005907,0.001346,0.003734,0.001171,0.002812,0.000071,...,0.000004,0.000947,0.000060,0.000712,0.000083,0.005667,0.001271,0.003538,0.001125,blank sol
2016-05-14 12:44:00,0,0,3.531424,2.753720,18.878641,4.908248,13.657210,4.267418,7.979821,0.182821,...,0.000121,3.523567,0.000460,2.747889,0.000428,18.837542,4.896816,13.624667,4.257370,blank sol
2016-05-14 12:56:00,0,0,0.000574,0.000430,0.003702,0.000786,0.002184,0.000691,0.001886,0.000038,...,0.000015,0.000599,0.000062,0.000433,0.000073,0.003782,0.000797,0.002241,0.000717,blank sol
2016-05-14 13:34:00,0,0,0.000774,0.000571,0.004698,0.001031,0.002914,0.000929,0.002293,0.000053,...,0.000006,0.000750,0.000044,0.000550,0.000070,0.004547,0.000996,0.002811,0.000902,blank sol
2016-05-14 14:30:00,0,0,0.000966,0.000727,0.005731,0.001322,0.003686,0.001165,0.002766,0.000052,...,0.000006,0.000776,0.000058,0.000572,0.000076,0.004693,0.001053,0.002936,0.000932,blank sol
2016-05-14 14:54:00,0,0,0.000710,0.000550,0.004439,0.000971,0.002756,0.000866,0.002191,0.000047,...,0.000011,0.000699,0.000045,0.000521,0.000068,0.004316,0.000957,0.002660,0.000829,blank sol
2016-05-14 15:17:00,0,0,0.001101,0.000847,0.006515,0.001530,0.004270,0.001343,0.003045,0.000066,...,-0.000030,0.001130,-0.000007,0.000862,0.000063,0.006670,0.001560,0.004358,0.001393,blank sol
2016-05-14 15:27:00,0,0,0.000586,0.000431,0.003718,0.000793,0.002237,0.000704,0.001884,0.000052,...,0.000024,0.000649,0.000053,0.000496,0.000073,0.004069,0.000874,0.002478,0.000788,blank sol
2016-05-14 16:05:00,0,0,0.000722,0.000516,0.004374,0.000984,0.002738,0.000850,0.002173,0.000058,...,0.000019,0.000749,0.000046,0.000578,0.000055,0.004640,0.001053,0.002933,0.000928,blank sol


In [11]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [30]:
df_all.to_csv(path_all + "blanks.csv", mode='w', header=True)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")